In [35]:
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam ,RMSprop
from keras import backend as K
from keras.callbacks import ModelCheckpoint
K.set_image_data_format('channels_last')
import numpy as np

In [36]:
labels = pd.read_csv('input/labels.csv')
imgs = []
Y = labels['breed']
encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)
Y_categorical = to_categorical(Y_encoded)

NUM_CLASSES = 120
IMAGE_SIZE = 128

print("Loading training images ...")
for key, value in labels.iterrows():
    image = load_img('input/train/{0}.jpg'.format(value['id']), target_size=(IMAGE_SIZE, IMAGE_SIZE))
    image = img_to_array(image)
    imgs.append(image/255.0)
print("Loading training images ... COMPLETE!")

X = np.array(imgs, dtype='float32')

Loading training images ...
Loading training images ... COMPLETE!


In [37]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(120, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def mini_alex_net():
    model = Sequential()
    mode.add()


In [4]:
# build the model
model = larger_model()
# Fit the model
model.fit(X, Y_categorical, epochs=100, batch_size=200, verbose=1, validation_split=0.2,callbacks=[ModelCheckpoint('model.h5', save_best_only=True)])

Train on 8177 samples, validate on 2045 samples
Epoch 1/100
8177/8177 [==============================] - 10s 1ms/step - loss: 4.7885 - acc: 0.0100 - val_loss: 4.7852 - val_acc: 0.0108
Epoch 2/100
8177/8177 [==============================] - 5s 650us/step - loss: 4.7709 - acc: 0.0147 - val_loss: 4.7660 - val_acc: 0.0142
Epoch 3/100
8177/8177 [==============================] - 5s 653us/step - loss: 4.6901 - acc: 0.0241 - val_loss: 4.6890 - val_acc: 0.0220
Epoch 4/100
8177/8177 [==============================] - 5s 652us/step - loss: 4.4464 - acc: 0.0473 - val_loss: 4.6294 - val_acc: 0.0274
Epoch 5/100
8177/8177 [==============================] - 6s 677us/step - loss: 4.1064 - acc: 0.0933 - val_loss: 4.6035 - val_acc: 0.0323
Epoch 6/100
8177/8177 [==============================] - 5s 646us/step - loss: 3.7170 - acc: 0.1476 - val_loss: 4.7157 - val_acc: 0.0372
Epoch 7/100
8177/8177 [==============================] - 5s 647us/step - loss: 3.2254 - acc: 0.2381 - val_loss: 5.0592 - val_acc: 0

Epoch 60/100
8177/8177 [==============================] - 5s 652us/step - loss: 0.0400 - acc: 0.9887 - val_loss: 12.5844 - val_acc: 0.0406
Epoch 61/100
8177/8177 [==============================] - 5s 641us/step - loss: 0.0332 - acc: 0.9900 - val_loss: 12.5338 - val_acc: 0.0406
Epoch 62/100
8177/8177 [==============================] - 5s 645us/step - loss: 0.0356 - acc: 0.9900 - val_loss: 12.4933 - val_acc: 0.0391
Epoch 63/100
8177/8177 [==============================] - 5s 645us/step - loss: 0.0299 - acc: 0.9928 - val_loss: 12.8640 - val_acc: 0.0411
Epoch 64/100
8177/8177 [==============================] - 5s 646us/step - loss: 0.0317 - acc: 0.9918 - val_loss: 12.6302 - val_acc: 0.0421
Epoch 65/100
8177/8177 [==============================] - 5s 649us/step - loss: 0.0311 - acc: 0.9906 - val_loss: 12.7065 - val_acc: 0.0406
Epoch 66/100
8177/8177 [==============================] - 5s 650us/step - loss: 0.0401 - acc: 0.9881 - val_loss: 12.5833 - val_acc: 0.0406
Epoch 67/100
8177/8177 [===

In [10]:
X_test_imgs = []
X_test_ids = []
import os
for filename in os.listdir('input/test'):
    image = load_img('input/test/{0}'.format(filename), target_size=(IMAGE_SIZE, IMAGE_SIZE))
    image = img_to_array(image)
    X_test_imgs.append(image/255.0)
    X_test_ids.append(filename.split('.')[0])

X_test = np.array(X_test_imgs, dtype='float32')

In [14]:
pred = model.predict(X_test)

In [16]:
pred[0].shape

(120,)

### Try out Data Augmentation

Since we have less amount of training data, we'll modify(rotate, bend, zoom etc.) the existing data to create more training data


In [38]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y_categorical,
                                                  test_size=0.2, random_state=42)

datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.)

datagen.fit(X_train)

In [40]:
epochs = 100
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=200),
                    steps_per_epoch=X_train.shape[0],
                    epochs=epochs,
                    validation_data=(X_val, Y_val),
                    callbacks=[ModelCheckpoint('model_augmented.h5', save_best_only=True)]
                    )

Epoch 1/30
   4/8177 [..............................] - ETA: 27:11:08 - loss: 4.7858 - acc: 0.0137

KeyboardInterrupt: 